## Python Calls

In [3]:
def julian(name):
    return(name+" Julian")

greet_julian = julian
print(greet_julian("Hello"))

Hello Julian


### Nested Functions

In [6]:
def foo():
    def bar():
        print("bar")
    bar()

In [7]:
foo()

bar


### Functions are parameters

In [6]:
def call_func(func):
    message = 'Calling'
    return func(message)

In [7]:
print(call_func(julian))

Calling Julian


In [11]:
def compose_call_res():
    def greet():
        return("Hello")
    return greet

In [12]:
greet = compose_call_res()
print(greet())

Hello who is this?


### Enclosing Scope

In [13]:
def compose_call_res(msg):
    def greet():
        return("Hello "+msg)
    return greet

In [16]:
fedex_greet = compose_call_res("FedEx")
print(fedex_greet())

Hello FedEx


### Function Wrappers

In [8]:
def get_text(name):
   return "Welcome to {0}".format(name)

def p_decorate(func):
   def func_wrapper(name):
       return "<title>{0}</tittle>".format(func(name))
   return func_wrapper

my_get_text = p_decorate(get_text)

print(my_get_text("CS3"))

<title>Welcome to CS3</tittle>


In [9]:
def p_decorate(func):
   def func_wrapper(name):
       return "<p>{0}</p>".format(func(name))
   return func_wrapper

def strong_decorate(func):
    def func_wrapper(name):
        return "<strong>{0}</strong>".format(func(name))
    return func_wrapper

def div_decorate(func):
    def func_wrapper(name):
        return "<div>{0}</div>".format(func(name))
    return func_wrapper

In [20]:
get_text = div_decorate(p_decorate(strong_decorate(get_text)))

In [21]:
print(get_text)

<function div_decorate.<locals>.func_wrapper at 0x7f41bde8cae8>


### Python Decorators

In [11]:
@div_decorate
@p_decorate
@strong_decorate
def get_text(name):
   return "Welcome to {0}".format(name)

print(get_text("CS3"))

<div><p><strong>Welcome to CS3</strong></p></div>


In [27]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)

In [30]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


In [12]:
def tags(tag_name):
    def tags_decorator(func):
        def func_wrapper(name):
            return "<{0}>{1}</{0}>".format(tag_name, func(name))
        return func_wrapper
    return tags_decorator

@tags("title")
def get_text(name):
    return "Hello "+name

print(get_text("Julian"))

<title>Hello Julian</title>


In [31]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

In [35]:
@do_twice
def say_whee(msg):
    print("Whee! "+msg)

In [36]:
say_whee("sdlkfjaskldfj")

Whee! sdlkfjaskldfj
Whee! sdlkfjaskldfj


In [48]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return(f"Hi {name}")
hi_adam = return_greeting("Adam")
print(hi_adam)

Creating greeting
Creating greeting
Hi Adam


In [44]:
hi_adam == None

True

In [45]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [49]:
hi_adam = return_greeting("Adam")

Creating greeting
Creating greeting


In [50]:
print(hi_adam)

Hi Adam


```Python
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator
```

In [13]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [14]:
waste_some_time(1)

Finished 'waste_some_time' in 0.0035 secs


In [17]:
import functools
import time

def slow_down(func):
    """Sleep 1 second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

In [18]:
countdown(3)

3
2
1
Liftoff!


In [56]:
import random
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)


In [57]:
PLUGINS

{'say_hello': <function __main__.say_hello(name)>,
 'be_awesome': <function __main__.be_awesome(name)>}

### Veridic Functions

```python
def name(_func=None, *, kw1=val1, kw2=val2, ...):  # 1
    def decorator_name(func):
        ...  # Create and return a wrapper function.

    if _func is None:
        return decorator_name                      # 2
    else:
        return decorator_name(_func)               # 3
    ```

In [58]:
def repeat(_func=None, *, num_times=2):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat

    if _func is None:
        return decorator_repeat
    else:
        return decorator_repeat(_func)


In [59]:
@repeat
def say_whee():
    print("Whee!")

@repeat(num_times=3)
def greet(name):
    print(f"Hello {name}")

In [60]:
say_whee()

Whee!
Whee!


In [61]:
greet("Penny")

Hello Penny
Hello Penny
Hello Penny


In [20]:
import functools

class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        #print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

In [21]:

@CountCalls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

In [22]:
fibonacci(10)

55

In [23]:
def set_unit(unit):
    """Register a unit on a function"""
    def decorator_set_unit(func):
        func.unit = unit
        return func
    return decorator_set_unit

In [24]:
import math

@set_unit("cm^3")
def volume(radius, height):
    return math.pi * radius**2 * height

In [25]:
volume(3, 5)

141.3716694115407

In [26]:
volume.unit

'cm^3'

In [77]:
def set(unit):
    """Register a unit on a function"""
    def decorator_set_unit(func):
        return func(unit,33)
    return decorator_set_unit

In [78]:
@set("cm^3")
def volume(radius, height):
    return math.pi * radius**2 * height

TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'